# Downloading genbank accessions from .xlsx table

In [123]:
import pandas as pd
from Bio import Entrez, SeqIO
import os, glob

In [124]:
spp_accession = pd.read_excel("spp_accession.xlsx", index_col=0)

In [125]:
spp_accession

,18S,28S,abdA,EF1,EF2,LWR,WNT
Species,,,,,,,
Formica_fusca,FJ824197.1,FJ824242.1,FJ824331.1,AY334415.1,EF013263.1,KJ920611.1,LT977286.1
Linepithema_humile,AH012894.2,AH012910.2,AY525327.1,EF013277.2,EF013277.2,FJ161830.1,AY233596.1
Pseudomyrmex_gracilis,AH012887.2,AH012903.2,DQ352945.1,NaN,NaN,KR829388.1,KR829015.1
Solenopsis_invicta,AY334566.1,AY325955.1,DQ352941.1,KP730149.1,EF013348.1,EF013642.1,KP730228.1


In [126]:
# Changing NaN to None
spp_accession = spp_accession.where(pd.notnull(spp_accession), None)

In [127]:
spp_accession

,18S,28S,abdA,EF1,EF2,LWR,WNT
Species,,,,,,,
Formica_fusca,FJ824197.1,FJ824242.1,FJ824331.1,AY334415.1,EF013263.1,KJ920611.1,LT977286.1
Linepithema_humile,AH012894.2,AH012910.2,AY525327.1,EF013277.2,EF013277.2,FJ161830.1,AY233596.1
Pseudomyrmex_gracilis,AH012887.2,AH012903.2,DQ352945.1,None,None,KR829388.1,KR829015.1
Solenopsis_invicta,AY334566.1,AY325955.1,DQ352941.1,KP730149.1,EF013348.1,EF013642.1,KP730228.1


In [128]:
def create_dir(dir_name):
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)

In [129]:
def download_acc_from_dataframe(dataframe):
    for species in dataframe.index:
        species = species.replace(" ", "_")
        for gene, accession in dataframe.loc[species].iteritems():
            gene_dir = "seed_fasta/{}/".format(gene)
            #create_dir(gene_dir)
            if accession:
                fasta_path = "seed_fasta/{}/{}-{}.fasta".format(gene,species, gene)
                create_dir(fasta_path)
                with open(fasta_path, "w") as fasta:
                    handle = Entrez.efetch(db='nucleotide', id=accession, rettype='fasta', retmode='text')
                    content = handle.read()
                    for line in content.split("\n"):
                        if line.startswith(">"):
                            line = ">{}-{} {}".format(species, gene, accession)
                        fasta.write("{}\n".format(line))
               # with open(fasta_path, "w") as fasta:
               #     handle = Entrez.efetch(db='nucleotide', id=accession, rettype='fasta', retmode='text')
               #     fasta.write(handle.read())
               # with open(fasta_path, "w")
               #     seqdata = SeqIO.read(fasta, "fasta")
               #     print(">{}-{}\n{}\n".format(species, gene, seqdata.seq))

In [130]:
def download_acc_from_dataframe(dataframe):
    for species in dataframe.index:
        species = species.replace(" ", "_")
        for gene, accession in dataframe.loc[species].iteritems():
            gene_dir = "seed_fasta/{}/".format(gene)
            #create_dir(gene_dir)
            if accession:
                fasta_path = "seed_fasta/{}/{}-{}.fasta".format(gene,species, gene)
                create_dir(fasta_path)
                with open(fasta_path, "w") as fasta:
                    handle = Entrez.efetch(db='nucleotide', id=accession, rettype='fasta', retmode='text')
                    content = handle.read()
                    for line in content.split("\n"):
                        if line.startswith(">"):
                            line = ">{}-{} {}".format(species, gene, accession)
                        fasta.write("{}\n".format(line))

In [131]:
download_acc_from_dataframe(spp_accession)

/home/gabriel/.local/lib/python3.8/site-packages/Bio/Entrez/__init__.py:619: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


In [140]:
def extract_COI(gb_file):
    for record in SeqIO.parse(gb_file, "genbank"):
        species_name = record.annotations.get('organism').replace(" ", "_")
        filename = "./seed_fasta/COX1/{}_coi.fasta".format(species_name)
        create_dir(filename)
        with open(filename, "w") as coi_file:
            for gene in record.features:
                if gene.type in ["CDS"] and gene.qualifiers.get('gene')[0] in ['COX1', 'COI']:
                    header = "{}-{}".format(species_name, gene.qualifiers.get('gene')[0])
                    sequence = gene.location.extract(record.seq) # Mas seq pode ter stop codon truncado
                    if len(sequence) % 3 == 1:
                        sequence += "AA" #Resto 1 - Precisa adicionar 'AA'
                    elif len(sequence) % 3 == 2:
                        sequence += "A" #Resto 2 - Precisa adicionar 'A'
                    coi_file.write(">{}\n{}\n".format(header, sequence))

In [141]:
def extract_CYTB(gb_file):
    for record in SeqIO.parse(gb_file, "genbank"):
        species_name = record.annotations.get('organism').replace(" ", "_")
        filename = "./seed_fasta/CYTB/{}_cytb.fasta".format(species_name)
        create_dir(filename)
        with open(filename, "w") as cytb_file:
            for gene in record.features:
                if gene.type in ["CDS"] and gene.qualifiers.get('gene')[0] in ['CYTB']:
                    header = "{}-{}".format(species_name, gene.qualifiers.get('gene')[0])
                    sequence = gene.location.extract(record.seq) # Mas seq pode ter stop codon truncado
                    if len(sequence) % 3 == 1:
                        sequence += "AA" #Resto 1 - Precisa adicionar 'AA'
                    elif len(sequence) % 3 == 2:
                        sequence += "A" #Resto 2 - Precisa adicionar 'A'
                    cytb_file.write(">{}\n{}\n".format(header, sequence))

In [142]:
def extract_12S(gb_file):
    for record in SeqIO.parse(gb_file, "genbank"):
        species_name = record.annotations.get('organism').replace(" ", "_")
        filename = "./seed_fasta/12S/{}_12s.fasta".format(species_name)
        create_dir(filename)
        with open(filename, "w") as coi_file:
            for gene in record.features:
                if gene.type in ["rRNA"] and gene.qualifiers.get('product')[0] in ['small subunit ribosomal RNA', 's-rRNA']:
                    header = "{}-12S".format(species_name)
                    sequence = gene.location.extract(record.seq)
                    coi_file.write(">{}\n{}\n".format(header, sequence))

In [143]:
for gb_file in glob.glob("./ant_mitogenomes/*.gb"):
    extract_COI(gb_file)
    extract_CYTB(gb_file)
    extract_12S(gb_file)

In [136]:
# Testing how to extract species, gene and acession at once from dataframe
#for species in spp_accession.index:
#    for gene, accession in spp_accession.loc[species].iteritems():
#        print(species, gene, accession)